# Binary data without lazy loading


**Author**: Aureliana Barghini ([B-Open](https://www.bopen.eu/))

## BackendEntrypoint
Implement a subclass of `BackendEntrypoint` that expose a method `open_dataset`:

```python
from xarray.backends import BackendEntrypoint

class MyBackendEntrypoint(BackendEntrypoint):
    def open_dataset(
        self,
        filename_or_obj,
        *,
        drop_variables=None,
    ):

        return my_open_dataset(filename_or_obj, drop_variables=drop_variables)

```

## BackendEntrypoint integration
Declare this class as an external plugin in your `setup.py`:

```python
setuptools.setup(
    ...
    entry_points={
        'xarray.backends': ['engine_name=package.module:my_backendentrypoint'],
    },
)
```
or pass it in `xr.open_dataset`:

```python
xr.open_dataset(..., engine=MyBackendEntrypoint)
```

## Example backend for binary files

In [ ]:
import numpy as np
import xarray as xr

### Create sample files

In [ ]:
arr = np.arange(30000000, dtype=np.int64)
with open("foo.bin", "w") as f:
    arr.tofile(f)

arr = np.arange(30000000, dtype=np.float64)
with open("foo_float.bin", "w") as f:
    arr.tofile(f)

### Define the entrypoint
Example of backend to open binary files

In [ ]:
class BinaryBackend(xr.backends.BackendEntrypoint):
    def open_dataset(
        self,
        filename_or_obj,
        *,
        drop_variables=None,
        # backend specific parameter
        dtype=np.int64,
    ):
        with open(filename_or_obj) as f:
            arr = np.fromfile(f, dtype)

        var = xr.Variable(dims=("x"), data=arr)
        coords = {"x": np.arange(arr.size) * 10}
        return xr.Dataset({"foo": var}, coords=coords)

### It Works! 
But it may be memory demanding

In [ ]:
arr = xr.open_dataarray("foo.bin", engine=BinaryBackend)
arr

In [ ]:
arr = xr.open_dataarray("foo_float.bin", engine=BinaryBackend, dtype=np.float64)
arr

In [ ]:
arr.sel(x=slice(0, 100))